In [1]:
from typing import Optional, Any
import json
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

In [2]:
# Some pointers on working with categorical data
# https://www.kaggle.com/getting-started/27270

In [3]:
GET, SET, UNKNOWN_STR = 'GET', 'SET', 'UNKNOWN'

JSON_PATH = Path('normalisation_data.json')
TRAIN_PATH = Path('data/train.csv')
TEST_PATH = Path('data/test.csv')
CSV_PREDS_OUT = Path('preds.csv')

## Load Data

In [4]:
df_test_raw = pd.read_csv(TEST_PATH)

df_train_raw = pd.read_csv(TRAIN_PATH)
df_train_raw.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",...,73000.0,45330.0,50462.0,NaN,NaN,uncategorized,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",NaN,71300.0
1,1021510,Toyota Hiace 3.0M,NaN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,...,110112.0,27502.0,1376.0,NaN,25-jan-2035,uncategorized,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,NaN,43800.0
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",...,80000.0,27886.0,26041.0,NaN,NaN,uncategorized,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,NaN,95500.0
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",...,9800.0,46412.0,56977.0,NaN,NaN,uncategorized,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,NaN,197900.0
4,1031014,Honda Civic 1.6A VTi,NaN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,...,40000.0,20072.0,20101.0,NaN,NaN,uncategorized,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",NaN,103200.0


In [15]:
print(list(df_train_raw.columns))

['listing_id', 'title', 'make', 'model', 'description', 'manufactured', 'original_reg_date', 'reg_date', 'type_of_vehicle', 'category', 'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap', 'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value', 'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category', 'features', 'accessories', 'indicative_price', 'price']


* Saurabh: title, make, model, description, features, accessories
* Kanav and Soeren: Handle missing values for numerical columns
* Soeren: type_of_vehicle, no_of_owners (see if in description)
* Kanav: category
* Dhawal: original_reg_date, reg_date, lifespan, fuel_type, opc_scheme

In [33]:
df_train_raw['description'][0]

'1 owner! 320i gt m-sports model! big brake kit, m-sport steering wheel! very well taken care of by ex-owner! in elegant white! big boot space as well! immaculate showroom condition! special promotional bank interest rates in house finance available with competitive interest rates. call today to enquire!'

In [47]:
df_train_raw['price']

0         71300.0
1         43800.0
2         95500.0
3        197900.0
4        103200.0
           ...   
16779    144400.0
16780     70200.0
16781     71300.0
16782     81200.0
16783    638000.0
Name: price, Length: 16784, dtype: float64

## Define functions for cleaning data

In [5]:
def get_or_set_val(col_name, key, value=None) -> Any:
    """
    Sets or gets a value from JSON file for a given column and key
    If value is supplied, it will set that value - otherwise it
    will try to get it. For sanity, no key can be set to a value of
    None (would not make sense for our problem anyway)
    """
    action = GET if value is None else SET
    prev_data = {}
    try:
        with open(JSON_PATH, 'r') as json_file:
            prev_data = json.load(json_file)
    except Exception as e:
        if action == GET:
            raise Exception(f'Cannot read file: {e}')

    # Get and return value
    if action == GET:
        return prev_data[col_name][key]

    # Update previous values with new (key, val) and dump to file again
    with open(JSON_PATH, 'w') as json_file:
        new_data = {
            **prev_data,
            col_name: {
                **prev_data.get(col_name, {}),
                key: value
            }
        }
        json.dump(new_data, json_file, indent=4)

    return None


def nan_to_str_fallback(value: Optional[str]) -> str:
    """
    Returns the input value if it string, otherwise 'unknown'
    Useful for columns that might contain NaN
    """
    return value if isinstance(value, str) else UNKNOWN_STR


TRANSMISSION_MAP = {
    'auto': 0,
    'manual': 1
}


FUEL_TYPE_MAP = {
    'electric': 0.0,
    'petrol-electric': 0.5,
    UNKNOWN_STR: 0.7,  # Arbitrarily chosen value
    'diesel': 1.0,
    'petrol': 1.0
}
    
    
def clean_number_col(df: pd.DataFrame,
                     col_name: str,
                     strategy: str = 'median',
                     outlier_mult: float = 0.2,
                     use_previous_vals: bool = False) -> pd.Series:
    """
    General function for cleaning a column of numerical values
    The overall logic is:
    1. Ignore all NaN
    2. Find 2th and 98th quantiles based on remaning values
    3. Set anything that is respectively below/above this to its nearest quantile
    4. Compute measure of location (median/avg/etc) based on these values
    5. Set all NaN to measure of location
    
    If this is run test set, `use_previous_vals` should be True to use
    the 2th and 98th quantile and measure of location computed for
    train set
    """
    col_raw = df[col_name]

    # Remove NaN
    col_out = col_raw.copy()
    self_diff = col_raw - col_raw
    is_non_nan = self_diff < 1e-10
    
    # Find outliers
    quantile_2_cmp, quantile_98_cmp = None, None
    if use_previous_vals:
        quantile_2_cmp = get_or_set_val(col_name, 'quantile_2_cmp')
        quantile_98_cmp = get_or_set_val(col_name, 'quantile_98_cmp')
    else:
        quantile_2, quantile_98 = col_raw[is_non_nan].quantile([0.02, 0.98])
        quantile_diff_mult = (quantile_98 - quantile_2) * outlier_mult
        quantile_2_cmp = quantile_2 - quantile_diff_mult
        quantile_2_cmp = max(quantile_2_cmp, col_raw[is_non_nan].min())
        quantile_98_cmp = quantile_98 + quantile_diff_mult
        quantile_98_cmp = min(quantile_98_cmp, col_raw[is_non_nan].max())
        
        get_or_set_val(col_name, 'quantile_2_cmp', value=quantile_2_cmp)
        get_or_set_val(col_name, 'quantile_98_cmp', value=quantile_98_cmp)
    
    # Set outliers to min/max
    col_out[col_raw > quantile_98_cmp] = quantile_98_cmp
    col_out[col_raw < quantile_2_cmp] = quantile_2_cmp
    
    # Find measure of location based on some strategy
    measure_of_location = None
    if strategy == 'median':
        measure_of_location = col_out[is_non_nan].median()
    elif strategy == 'mean':
        measure_of_location = col_out[is_non_nan].mean()
    elif strategy == 'max':
        measure_of_location = col_out[is_non_nan].max()
    elif strategy == 'min':
        measure_of_location = col_out[is_non_nan].min()
    else:
        raise ValueError(f'Strategy {strategy} not known')
        
    mol_key = f'mol-{strategy}'
    if use_previous_vals:
        measure_of_location = get_or_set_val(col_name, mol_key)
    else:
        get_or_set_val(col_name, mol_key, value=measure_of_location)

    # Set all NaN to measure of location
    col_out[~is_non_nan] = measure_of_location

    return col_out

In [6]:
STR_COLS = [
    'make',
    'title',
    'model',
    'description',
    'type_of_vehicle',
    'category',
    'fuel_type',
    'opc_scheme',
    'features',
    'accessories'
]

NUMBER_COLS = [
    'manufactured',
    'road_tax',
    'curb_weight',
    'power',
    'engine_cap',
    'no_of_owners',
    'depreciation',
    'coe',
    'road_tax',
    'dereg_value',
    'mileage',
    'omv',
    'arf'
]


def clean_df(df_raw: pd.DataFrame, is_test: bool = False):
    """
    Cleans a DataFrame
    """
    df_clean = df_raw.copy()

    for str_col in STR_COLS:
        df_clean[str_col] = df_clean[str_col].apply(nan_to_str_fallback)

    for number_col in NUMBER_COLS:
        df_clean[number_col] = clean_number_col(df_clean, number_col, use_previous_vals=is_test)

    df_clean['fuel_type'] = df_clean['fuel_type'].map(FUEL_TYPE_MAP)
    df_clean['transmission'] = df_clean['transmission'].map(TRANSMISSION_MAP)

    # The following two are either all NaN or all the same value for all rows
    to_drop = ['indicative_price', 'eco_category']
    df_clean.drop(to_drop, axis=1, inplace=True, errors='ignore')

    if not is_test:
        df_clean['price'] = df_clean['price'].apply(round)
        
    return df_clean

## Run cleaning

In [7]:
df_train_clean = clean_df(df_train_raw)
df_test_clean = clean_df(df_test_raw, is_test=True)
df_train_clean

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories,price
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",...,1210.0,47514.000,73000.0,45330.0,50462.00,UNKNOWN,NaN,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",71300
1,1021510,Toyota Hiace 3.0M,UNKNOWN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,...,976.0,3648.000,110112.0,27502.0,1376.00,UNKNOWN,25-jan-2035,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,43800
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",...,740.0,44517.000,80000.0,27886.0,26041.00,UNKNOWN,NaN,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,95500
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",...,684.0,80301.000,9800.0,46412.0,56977.00,UNKNOWN,NaN,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,197900
4,1031014,Honda Civic 1.6A VTi,UNKNOWN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,...,742.0,36453.000,40000.0,20072.0,20101.00,UNKNOWN,NaN,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",103200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16779,1030181,Volvo XC90 T5 Momentum,volvo,xc90,"just arrived, serviced and maintained by wearn...",2015.0,NaN,22-jan-2016,suv,"parf car, premium ad car",...,1176.0,77443.000,64000.0,56261.0,73270.00,UNKNOWN,NaN,"250bhp, 350nm, 4 cylinder inline 16 valve turb...","city safety with pedestrian/cyclist detection,...",144400
16780,1027041,Hyundai Ioniq Hybrid 1.6A DCT Sunroof,hyundai,ioniq,one owner unit! 10 years hybrid battery till 0...,2017.0,NaN,11-may-2017,hatchback,"parf car, premium ad car, hybrid cars",...,732.0,34709.000,100808.0,27752.0,5000.00,UNKNOWN,NaN,"1.6l hybrid dohc inline-4, 16v dual-cvvt, 138b...","17"" rims, leather upholstery, reverse camera/s...",70200
16781,1021099,MINI Cooper Cabriolet 1.6A (COE till 08/2030),mini,cooper,all wear and tear has been done up with receip...,2010.0,NaN,27-aug-2010,sports car,"coe car, premium ad car",...,816.0,28997.000,65801.5,23762.0,23762.00,UNKNOWN,NaN,1.6l responsive turbocharged engine. 6 speed a...,multifunction steering. auto headlights. rever...,71300
16782,1019473,Honda Fit 1.3A G F-Package,UNKNOWN,fit,"1 owner unit, 100% loan available! well-mainta...",2019.0,NaN,30-jul-2020,hatchback,"parf car, premium ad car",...,578.0,33506.000,13000.0,15734.0,5734.00,UNKNOWN,NaN,"4 cylinders dohc i-vtec engine, 97 bhp, cvt au...","sports rims, leather seats, reverse sensors, r...",81200


In [8]:
X_train = df_train_clean[NUMBER_COLS]
y_train = df_train_clean['price']

In [9]:
X_train

,manufactured,road_tax,curb_weight,power,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf
0,2013.0,1210.0,1560.0,135.0,1997.0,1.0,17700.0,77100.0,1210.0,47514.000,73000.0,45330.0,50462.00
1,2014.0,976.0,1740.0,110.0,2982.0,3.0,11630.0,10660.0,976.0,3648.000,110112.0,27502.0,1376.00
2,2016.0,740.0,1430.0,90.0,1595.0,1.0,15070.0,53694.0,740.0,44517.000,80000.0,27886.0,26041.00
3,2019.0,684.0,1635.0,115.0,1497.0,1.0,16400.0,40690.0,684.0,80301.000,9800.0,46412.0,56977.00
4,2019.0,742.0,1237.0,92.0,1597.0,1.0,10450.0,26667.0,742.0,36453.000,40000.0,20072.0,20101.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16779,2015.0,1176.0,2053.0,187.0,1969.0,2.0,21720.0,60001.0,1176.0,77443.000,64000.0,56261.0,73270.00
16780,2017.0,732.0,1370.0,103.6,1580.0,1.0,10770.0,54405.0,732.0,34709.000,100808.0,27752.0,5000.00
16781,2010.0,816.0,1205.0,88.0,1598.0,5.0,7190.0,32190.0,816.0,28997.000,65801.5,23762.0,23762.00
16782,2019.0,578.0,1030.0,73.0,1317.0,1.0,7940.0,32699.0,578.0,33506.000,13000.0,15734.0,5734.00


## Train linear regressor and predict

In [10]:
def rmse(*args, **kwargs):
    return mse(*args, **kwargs, squared=False)

In [11]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [12]:
reg = LinearRegression().fit(X_train_scaled, y_train)
preds_train = reg.predict(X_train_scaled)
print('rmse =', rmse(y_train, preds_train))

rmse = 63233.149129444486


In [13]:
X_test = df_test_clean[NUMBER_COLS]
X_test_scaled = scaler.transform(X_test)
preds = reg.predict(X_test_scaled)

In [14]:
def preds_to_csv(preds: np.ndarray) -> None:
    df = pd.DataFrame(preds, columns=['Predicted'])
    df.to_csv(CSV_PREDS_OUT, index_label='Id')
    
    
preds_to_csv(preds)